In [8]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pandas_market_calendars import get_calendar

In [2]:
polygon_api_key = "KkfCQ7fsZnx0yK4bhX9fD81QplTh0Pf3"
calendar = get_calendar("NYSE")


In [17]:
engine = sqlalchemy.create_engine("sqlite:///C:/Users/lewis/OneDrive/tradingstrategies/databases/momentum_strategy_database.db")
tickers = pd.read_sql("SELECT * FROM all_stocks", con = engine)["ticker"].values

In [23]:
dates = calendar.schedule(start_date = "2024-06-01", end_date = (datetime.today()-timedelta(days=1))).index.strftime("%Y-%m-%d").values

In [19]:
grouped_bars_list = []
times = []

In [24]:
for date in dates:
    
    start_time = datetime.now()
    
    grouped_bars_request = pd.json_normalize(requests.get(f"https://api.polygon.io/v2/aggs/grouped/locale/us/market/stocks/{date}?adjusted=true&apiKey={polygon_api_key}").json()["results"])
    grouped_bars_request = grouped_bars_request[grouped_bars_request["T"].isin(tickers)].copy()
    grouped_bars_list.append(grouped_bars_request)
    
    end_time = datetime.now()
    seconds_to_complete = (end_time - start_time).total_seconds()
    times.append(seconds_to_complete)
    iteration = round((np.where(dates==date)[0][0]/len(dates))*100,2)
    iterations_remaining = len(dates) - np.where(dates==date)[0][0]
    average_time_to_complete = np.mean(times)
    estimated_completion_time = (datetime.now() + timedelta(seconds = int(average_time_to_complete*iterations_remaining)))
    time_remaining = estimated_completion_time - datetime.now()
    
    print(f"{iteration}% complete, {time_remaining} left, ETA: {estimated_completion_time}")

          T          v       vw      o      c       h       l              t  \
0       DSX   847792.0   3.0456   3.02   3.05   3.065   3.010  1717444800000   
5       GMS   365226.0  92.4030  94.83  92.05  94.830  91.660  1717444800000   
7      AKAN   230939.0   2.6432   2.65   2.65   2.740   2.500  1717444800000   
9      CIVI  1987005.0  69.7015  73.79  69.63  73.790  68.675  1717444800000   
12     TSVT  1674413.0   4.1980   4.28   4.18   4.350   4.120  1717444800000   
...     ...        ...      ...    ...    ...     ...     ...            ...   
10208  BYNO      250.0  11.2708  11.26  11.26  11.260  11.260  1717444800000   
10209  MITA      567.0  11.0792  11.10  11.10  11.100  11.100  1717444800000   
10210  ERNA      590.0   1.9299   1.90   1.90   1.900   1.900  1717444800000   
10214  AFAR     2200.0  11.3294  11.33  11.33  11.330  11.330  1717444800000   
10217  RDIB      575.0  13.7927  13.53  13.53  13.530  13.530  1717444800000   

             n  
0       3926.0  
5    